In [43]:
import time
import urllib
import os.path
import sys
import os
import numpy as np
import matplotlib.pylab as plt
from os import listdir
import numpy as np
from PIL import Image

In [44]:
path = os.path.join(os.getcwd(), "archive\Training")
images = []
classifiers = []

In [45]:
gliomaPath = os.path.join(path, 'glioma_tumor')
files = os.listdir(gliomaPath)
for file in files:
    try:
        with Image.open(gliomaPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(1)
    except FileNotFoundException:
        print(file)
        break

In [46]:
meningPath = os.path.join(path, 'meningioma_tumor')
files = os.listdir(meningPath)
for file in files:
    try:
        with Image.open(meningPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(2)
    except FileNotFoundException:
        print(file)
        break

In [47]:
noTumorPath = os.path.join(path, 'glioma_tumor')
files = os.listdir(noTumorPath)
for file in files:
    try:
        with Image.open(noTumorPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(0)
    except FileNotFoundException:
        print(file)
        break

In [48]:
pituitaryPath = os.path.join(path, 'pituitary_tumor')
files = os.listdir(pituitaryPath)
for file in files:
    try:
        with Image.open(pituitaryPath + '\\' + file).convert('L') as image:
            image = image.resize((200,200))
            images.append(np.asarray(image).flatten())
            classifiers.append(3)
    except FileNotFoundException:
        print(file)
        break

(40000,)
